In [ ]:
#README

#below is the code for each aspect of the project
#each cell has a title describing it's purpose and as such how it should be executed

In [ ]:
#Obtaining replays - this code was used to create the dataset
#this does not need to be run as a hard copy already exists

#dates for past seasons obtained from https://www.esportstales.com/rocket-league/competitive-season-end-date

#API token unique to my ballchasing.com account
token = "rdJZjDqqQh5F8xUGbyo8crfZTCXVA3rPfdlGuyK5"

#base url used in API requests
base_url = "https://ballchasing.com/api"

#finding all the stats that haven't been averaged over the duration of the game and dividing them by the game
#time (in mins, follows since this is how boost is averaged and the comparison with other stats that are already)
#averaged is unchanged

from pandas import json_normalize
import time
import requests
import pandas as pd
import datetime

def sort_and_average(stats):
    players = []
    duration = stats['duration']
    for a in stats['blue']['players']:
        players.append(a['stats'])
    for b in stats['orange']['players']:
        players.append(b['stats'])
    key_stats = json_normalize(players)
    for s in key_stats.columns:
        if ("avg" in s) or ("percent" in s) or ("pm" in s) or ("mvp" in s):
            continue
        else:
            key_stats[s] /= (duration/60)
    for i in range(len(key_stats['core.mvp'])):
        key_stats.loc[i]['core.mvp'] = int(key_stats['core.mvp'][i])
    return key_stats


#function that returns the next date to look replays up until
def next_date(date, season):
    #list of dates 2 weeks after start of each season of 3 most recent complete seasons
    current_seas = season
    new_seas = season
    key_dates_str = ["2022-06-29", "2022-09-21", "2022-12-21"]
    key_dates = []
    #converting key_dates to datetime objects
    for d in key_dates_str:
        dmy = datetime.datetime(int(d[0:4]), int(d[5:7]), int(d[8:10]))
        key_dates.append(dmy)
    date_str = "T00:00:00Z"
    #baseline date according to the season being looked at
    baseline = key_dates[int(current_seas[1])-7]
    #converting date to datetime object
    date = datetime.datetime(int(date[0:4]), int(date[5:7]), int(date[8:10]))
    pos_date = date+datetime.timedelta(days=-18)
    #checking if new date falls within 2 weeks of start of season
    #if it does, reset dates to start looking for matches from end of previous season
    if (baseline>pos_date) == True:
        new_seas='f'+str(int(current_seas[1])-1)
        before_date = baseline+datetime.timedelta(days=-15)
        after_date = str(before_date+datetime.timedelta(days=-18))[:10]+date_str
        before_date = str(before_date)[:10]+date_str
        print(f"New season: {new_seas}")
    #if not, subtract 2 days from after date and input date becomes before date, season is unchanged
    else:
        before_date = str(date)[:10]+date_str
        after_date = str(pos_date)[:10]+date_str
    #returning new pair of dates along with the season being looked in 
    return before_date, after_date, new_seas

#date="2022-07-01"
#season='f9'
#print(date, next_date(date, season))

ranks_to_remove = ["bronze-1", "bronze-2", "bronze-3", "silver-1", "silver-2", "silver-3"]
ranks = ["platinum-1", "platinum-2", "platinum-3", "diamond-1", "diamond-2", "diamond-3", "champion-1", "champion-2", "champion-3", "grand-champion-1", "grand-champion-2", "grand-champion-3", "supersonic-legend"]
ranks_done = ["gold-1", "gold-2", "gold-3"]
dates_to_ignore = ["2020-12-23T00:00:00Z", "2021-04-07T00:00:00Z", "2021-04-21T00:00:00Z", "2021-08-11T00:00:00Z", "2021-08-25T00:00:00Z", "2021-11-17T00:00:00Z", "2021-12-01T00:00:00Z", "2022-03-09T00:00:00Z", "2022-03-23T00:00:00Z", "2022-06-15T00:00:00Z", "2022-06-29T00:00:00Z", "2022-09-07T00:00:00Z", "2022-09-21T00:00:00Z", "2023-01-01T00:00:00Z"]
replay_list = []
ranks_done = 0
season = 'f9'
starting_date = "2023-03-07T00:00:00Z" #final full day before season ended 

#main loop to create dataset
for rank in ranks:
    start = time.time()
    #creating list of ids to ensure not saving same replay from perspective of a different player
    ids_checked = []
    days_checked = 0
    main_calls=0
    sub_calls=0
    limit_reached=False
    before_date, after_date, season = next_date(starting_date, season)
    print("Starting new rank...")
    if ranks_done == len(ranks):
        print("All ranks completed.")
        break
    else:
        while limit_reached == False:
            time.sleep(2) #stops too many calls being made
            resp = requests.get(url=f"{base_url}/replays", headers={"Authorization": token}, params={"playlist": "ranked-standard", "min-rank": f"{rank}", "max-rank": f"{rank}", "replay-date-after": f"{after_date}", "replay-date-before": f"{before_date}", "count":"200"})
            if resp.status_code == 200:
                replays = resp.json()
                for rep in replays["list"]:
                    if sub_calls < 1000:
                        time.sleep(0.4)
                        di = rep['id']
                        if not (di in ids_checked):
                            ids_checked.append(di)
                            resp_2 = requests.get(url=f"{base_url}/replays/{di}", headers={"Authorization": token})
                            sub_calls+=1
                            if resp_2.status_code == 200:
                                resp_2 = resp_2.json()
                                dummy = sort_and_average(resp_2)
                                #removing any rows that contain NaN values
                                dummy = dummy.dropna()
                                #onyl adding to list if there are still valid rows
                                if dummy.shape[0] != 0:
                                    #replay is valid, add season column
                                    season_list = [season for i in range(dummy.shape[0])]
                                    dummy['season'] = season_list
                                    replay_list.append(dummy)
                    else:
                        print('reached limit')
                        limit_reached=True
                        break
            else:
                print(f"{resp.status_code}, call failed.")
                break
            #changing dates being searched between
            before_date, after_date, season = next_date(after_date, season)
            print(sub_calls)
        #saving replays to csv file
        replays_to_save = pd.concat(replay_list)
        replays_to_save.to_csv(f"MAIN_{rank}_replays.csv")
        print(f"{rank} done.")
        ranks_done+=1
        replay_list = []
        end = time.time()
        print(f"Took {(end-start)/60} mins.")
    time.sleep(3600-(end-start)) #pausing for an hour to stop too many calls being made
print("Finished.")

In [ ]:
#Machine learning code below

In [4]:
#this code loads the dataset for the machine learning algorithms and splits it into training and test sets
#execute this cell to load the dataset

#reading in and formatting replays

import pandas as pd
import csv
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics, preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb

ranks = ["gold-1", "gold-2", "gold-3", "platinum-1", "platinum-2", "platinum-3", "diamond-1", "diamond-2", "diamond-3", "champion-1", "champion-2", "champion-3", "grand-champion-1", "grand-champion-2", "grand-champion-3", "supersonic-legend"]
dif_ranks = ["gold-1", "gold-2", "gold-3", "diamond-1", "diamond-2", "diamond-3", "grand-champion-1", "grand-champion-2", "grand-champion-3"]
ranks_num = []
prop = 0.7
infile = open("MMR_branges.txt", "r+")
base_nums = []
for line in infile:
    base_nums.append(int((line.split('\n')[0]).split(',')[1]))
for i in range(len(base_nums)-1):
    ranks_num.append((base_nums[i]+base_nums[i+1])/2)
d = [i for i in range(len(ranks_num))]
a, b, c = np.polyfit(d, ranks_num, 2)
ranks_num.append(a*(len(ranks_num)**2)+b*len(ranks_num)+c)
d.append(len(ranks_num)-1)
r_tgt_train = pd.DataFrame([])
r_tgt_test = pd.DataFrame([])
c_tgt_train = pd.DataFrame([])
c_tgt_test = pd.DataFrame([])
feat_train = pd.DataFrame([])
feat_test = pd.DataFrame([])
sparse_feats = pd.DataFrame([])
sparse_tgts = pd.DataFrame([])

for r in ranks:
    temp_csv = pd.read_csv(f"MAIN_{r}_replays.csv", dtype=str)
    feat_temp = temp_csv.drop("Unnamed: 0", axis=1)
    feat_temp = feat_temp.dropna()
    #2986 is min number of replays for each rank, this line balances the dataset
    feat_temp = feat_temp[:2986]
    rows = len(feat_temp.index)
    r_list_train = pd.DataFrame([ranks_num[ranks.index(r)] for i in range(int(rows*prop))])
    r_list_test = pd.DataFrame([ranks_num[ranks.index(r)] for i in range(rows-int(rows*prop))])
    c_list_train = pd.DataFrame(r for i in range(int(prop*rows)))
    c_list_test = pd.DataFrame(r for i in range(rows-int(prop*rows)))
    feat_train_temp, feat_test_temp = train_test_split(feat_temp, train_size=prop)
    r_tgt_train = pd.concat([r_tgt_train, r_list_train])
    r_tgt_test = pd.concat([r_tgt_test, r_list_test])
    c_tgt_train = pd.concat([c_tgt_train, c_list_train])
    c_tgt_test = pd.concat([c_tgt_test, c_list_test])
    feat_train = pd.concat([feat_train, feat_train_temp])
    feat_test = pd.concat([feat_test, feat_test_temp])
    feat_train.reset_index(drop=True, inplace=True)
    feat_test.reset_index(drop=True, inplace=True)
    c_tgt_train.reset_index(drop=True, inplace=True)
    c_tgt_test.reset_index(drop=True, inplace=True)
    r_tgt_train.reset_index(drop=True, inplace=True)
    r_tgt_test.reset_index(drop=True, inplace=True)

for r in feat_train.index:
    feat_train['season'][r] = float(feat_train['season'][r][1])
    if 'T' in feat_train['core.mvp'][r]:
        feat_train.at[r, 'core.mvp']=1
    else:
        feat_train.at[r, 'core.mvp']=0
    
for r in feat_test.index:
    feat_test['season'][r] = float(feat_test['season'][r][1])
    if 'T' in feat_test['core.mvp'][r]:
        feat_test.at[r, 'core.mvp']=1
    else:
        feat_test.at[r, 'core.mvp']=0

feat_train = feat_train.astype(float)
feat_test = feat_test.astype(float)

#dropping features that don't provide any new information
bad_feats = ["boost.count_collected_big", "boost.count_stolen_big", "boost.count_collected_small", "boost.count_stolen_small", "boost.percent_zero_boost", "boost.percent_full_boost", "boost.percent_boost_0_25", "boost.percent_boost_25_50", "boost.percent_boost_50_75", "boost.percent_boost_75_100", "movement.avg_powerslide_duration", "movement.percent_slow_speed", "movement.percent_boost_speed", "movement.percent_supersonic_speed", "movement.percent_ground", "movement.percent_low_air", "movement.percent_high_air", "positioning.percent_defensive_third", "positioning.percent_offensive_third", "positioning.percent_neutral_third", "positioning.percent_defensive_half", "positioning.percent_offensive_half", "positioning.percent_behind_ball", "positioning.percent_infront_ball", "positioning.percent_most_back", "positioning.percent_most_forward", "positioning.percent_closest_to_ball", "positioning.percent_farthest_from_ball"]
feat_train.drop(columns=bad_feats, inplace=True)
feat_test.drop(columns=bad_feats, inplace=True)

In [2]:
feat_train

,core.shots,core.shots_against,core.goals,core.goals_against,core.saves,core.assists,core.score,core.mvp,core.shooting_percentage,boost.bpm,...,positioning.time_behind_ball,positioning.time_infront_ball,positioning.time_most_back,positioning.time_most_forward,positioning.goals_against_while_last_defender,positioning.time_closest_to_ball,positioning.time_farthest_from_ball,demo.inflicted,demo.taken,season
0,0.000000,1.840491,0.000000,0.368098,0.000000,0.000000,6.993865,0.0,0.000000,303.0,...,46.146626,13.976687,20.558282,17.852761,0.368098,20.208589,18.920245,0.000000,0.000000,9.0
1,0.641711,1.122995,0.320856,0.641711,0.320856,0.000000,78.288770,0.0,50.000000,318.0,...,40.868984,17.854011,16.203209,22.828877,0.320856,25.780749,17.823529,0.000000,0.320856,8.0
2,0.168067,0.168067,0.000000,0.168067,0.000000,0.000000,22.184874,0.0,0.000000,312.0,...,44.252101,15.687395,18.789916,20.403361,0.168067,17.210084,21.949580,0.000000,0.000000,9.0
3,0.810811,0.324324,0.648649,0.324324,0.000000,0.000000,101.675676,1.0,80.000000,244.0,...,45.415135,14.516757,28.167568,25.394595,0.162162,31.183784,22.767568,0.162162,0.000000,9.0
4,0.000000,0.718563,0.000000,0.179641,0.000000,0.359281,43.473054,0.0,0.000000,192.0,...,38.085629,21.917964,17.514970,24.179641,0.179641,22.544910,18.790419,0.000000,0.000000,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33435,0.000000,1.288344,0.000000,0.368098,0.000000,0.000000,27.239264,0.0,0.000000,455.0,...,37.157669,22.971166,15.515337,25.214724,0.184049,20.760736,18.128834,0.552147,0.000000,8.0
33436,0.776699,1.165049,0.582524,0.194175,0.194175,0.194175,131.650485,1.0,75.000000,432.0,...,48.011650,11.027184,26.019417,12.097087,0.194175,19.747573,15.941748,0.000000,0.194175,8.0
33437,0.425532,2.127660,0.141844,0.425532,0.283688,0.000000,60.992908,0.0,33.333332,405.0,...,40.903546,17.770213,19.361702,17.900709,0.141844,18.609929,15.120567,0.425532,0.425532,8.0
33438,0.380952,1.142857,0.000000,0.190476,0.190476,0.000000,29.714286,0.0,0.000000,370.0,...,39.135238,20.352381,19.180952,19.428571,0.190476,20.590476,18.171429,0.190476,0.190476,8.0


In [3]:
#this cell shows various different ideas experimented with which did not work

#since this is a recreation, this cell should not be run since it is unlikely to execute correctly and does not
#improve performance

import pandas as pd

#list of feats to remove that provide no new info and will likely confuse algorithms
bad_feats = ["boost.count_collected_big", "boost.count_stolen_big", "boost.count_collected_small", "boost.count_stolen_small", "boost.percent_zero_boost", "boost.percent_full_boost", "boost.percent_boost_0_25", "boost.percent_boost_25_50", "boost.percent_boost_50_75", "boost.percent_boost_75_100", "movement.avg_powerslide_duration", "movement.percent_slow_speed", "movement.percent_boost_speed", "movement.percent_supersonic_speed", "movement.percent_ground", "movement.percent_low_air", "movement.percent_high_air", "positioning.percent_defensive_third", "positioning.percent_offensive_third", "positioning.percent_neutral_third", "positioning.percent_defensive_half", "positioning.percent_offensive_half", "positioning.percent_behind_ball", "positioning.percent_infront_ball", "positioning.percent_most_back", "positioning.percent_most_forward", "positioning.percent_closest_to_ball", "positioning.percent_farthest_from_ball"]

#trying random combos to see which features are important

import random as rand

#splitting features into different categories

all_feats = list(feat_train.columns)
core_feats = []
boost_feats = []
move_feats = []
pos_feats = []
for f in all_feats:
    if f.split('.')[0] == 'boost':
        boost_feats.append(f)
    elif f.split('.')[0] == 'movement':
        move_feats.append(f)
    elif f.split('.')[0] == 'positioning':
        pos_feats.append(f)
    else:
        core_feats.append(f)
        

def rand_feats(n, t):
    feats_to_keep = []
    if t == 'b':
        while len(feats_to_keep) != n:
            r = rand.randint(0, len(boost_feats)-1)
            if not (boost_feats[r] in feats_to_keep):
                feats_to_keep.append(boost_feats[r])
    elif t == 'c':
        while len(inds) != n:
            r = rand.randint(0, len(core_feats))
            if not (r in inds):
                inds.append(r)
                feats_to_keep.append(core_feats[r])
    elif t == 'm':
        while len(inds) != n:
            r = rand.randint(0, len(move_feats))
            if not (r in inds):
                inds.append(r)
                feats_to_keep.append(move_feats[r])
    elif t == 'p':
        while len(inds) != n:
            r = rand.randint(0, len(pos_feats))
            if not (r in inds):
                inds.append(r)
                feats_to_keep.append(pos_feats[r])
    feats_to_keep = set(feats_to_keep)
    feats = set(all_feats)
    temp_feats = feats-feats_to_keep
    new_train_set = feat_train.drop(columns=temp_feats)
    new_test_set = feat_test.drop(columns=temp_feats)
    return new_train_set, new_test_set

lbest_set = []
lbest_acc = 0
tries = 100
for hmm in range(tries):
    train_fts, test_fts = rand_feats(5, 'b')
    lin_mod = LinearRegression().fit(train_fts, r_tgt_train)
    lin_pred_test = lin_mod.predict(test_fts)
    lin_train = round(r_predicts_to_acc(lin_pred_test, r_tgt_test)*100, 2)
    if lin_train > lbest_acc:
        lbest_acc = lin_train
        lbest_set = train_fts
print(lbest_acc)
print(lbest_set)

#shrinking dataset as much as possible
#keeping:
#boost: bpm, bcpm, avg am, am col big, am col smol, amount used ss, perc 0, perc 100, perc 0-25, perc 25-50, perc 50-75, perc 75-100
#
#

#removing irrelevant features

boost_feats_to_drop = ["boost.amount_collected", "boost.amount_stolen", "boost.amount_stolen_big", "boost.amount_stolen_small", "boost.count_collected_big", "boost.count_stolen_big", "boost.count_collected_small", "boost.count_stolen_small", "boost.amount_overfill", "boost.amount_overfill_stolen", "boost.time_zero_boost", "boost.time_full_boost", "boost.time_boost_0_25", "boost.time_boost_25_50", "boost.time_boost_50_75", "boost.time_boost_75_100"]
core_feats_to_drop = ["core.assists", "core.shots_against", "core.goals", "core.goals_against", "demo.inflicted", "demo.taken", "core.score"]
movement_feats_to_drop = ["movement.total_distance", "movement.time_supersonic_speed", "movement.time_boost_speed", "movement.time_slow_speed", "movement.time_ground", "movement.time_low_air", "movement.time_high_air", "movement.time_powerslide", "movement.count_powerslide", "movement.avg_speed_percentage"]
positioning_feats_to_drop = ["positioning.avg_distance_to_ball", "positioning.avg_distance_to_ball_possession", "positioning.avg_distance_to_ball_no_possession", "positioning.avg_distance_to_mates", "positioning.time_defensive_third", "positioning.time_neutral_third", "positioning.time_offensive_third", "positioning.time_defensive_half", "positioning.time_offensive_half", "positioning.time_behind_ball", "positioning.time_infront_ball", "positioning.time_most_back", "positioning.time_most_forward", "positioning.time_closest_to_ball", "positioning.time_farthest_from_ball", "positioning.percent_behind_ball", "positioning.percent_infront_ball", "positioning.percent_most_back", "positioning.percent_most_forward", "positioning.percent_closest_to_ball", "positioning.percent_farthest_from_ball"]

feat_train.drop(columns=boost_feats_to_drop+core_feats_to_drop+movement_feats_to_drop+positioning_feats_to_drop, inplace=True)
feat_test.drop(columns=boost_feats_to_drop+core_feats_to_drop+movement_feats_to_drop+positioning_feats_to_drop, inplace=True)

NameError: name 'r_predicts_to_acc' is not defined

In [ ]:
#code obtained from https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

#scaling and normalising the data

#scaling data
scaler = StandardScaler()
feat_train = scaler.fit_transform(feat_train)
feat_test = scaler.transform(feat_test)

In [ ]:
#dimensionality reduction
from sklearn.decomposition import PCA

#reducing the dimensionality of the data

pca = PCA(svd_solver='full')
feat_train = pca.fit_transform(feat_train)
feat_test = pca.transform(feat_test)

In [6]:
#algorithms to convert predictions to accuracies, should be run before any regression algorithms are trained

def ez_r_predicts_to_acc(pred, tgts, alg, data_type):
    all_cranks_train = []
    all_cranks_test = []
    all_cranks_train.append([0]*16)
    all_cranks_train.append([0]*16)
    all_cranks_train.append([0]*16)
    all_cranks_train.append([0]*16)
    all_cranks_train.append([0]*16)
    all_cranks_test.append([0]*16)
    all_cranks_test.append([0]*16)
    all_cranks_test.append([0]*16)
    all_cranks_test.append([0]*16)
    all_cranks_test.append([0]*16)
    corr = 0
    pred_cp = list(pred)
    tgt_test = list(tgts[0])
    for p in range(len(pred)):
        if pred[p] >= 475:
            if (pred[p] < 535):
                pred_cp[p] = ranks_num[0]
            elif (pred[p] < 594):
                pred_cp[p] = ranks_num[1]            
            elif (pred[p] < 655):
                pred_cp[p] = ranks_num[2]
            elif (pred[p] < 715):
                pred_cp[p] = ranks_num[3]
            elif (pred[p] < 774):
                pred_cp[p] = ranks_num[4]
            elif (pred[p] < 835):
                pred_cp[p] = ranks_num[5]
            elif (pred[p] < 915):
                pred_cp[p] = ranks_num[6]
            elif (pred[p] < 995):
                pred_cp[p] = ranks_num[7]
            elif (pred[p] < 1075):
                pred_cp[p] = ranks_num[8]
            elif (pred[p] < 1195):
                pred_cp[p] = ranks_num[9]
            elif (pred[p] < 1315):
                pred_cp[p] = ranks_num[10]
            elif (pred[p] < 1435):
                pred_cp[p] = ranks_num[11]
            elif (pred[p] < 1575):
                pred_cp[p] = ranks_num[12]
            elif (pred[p] < 1715):
                pred_cp[p] = ranks_num[13]
            elif (pred[p] < 1861):
                pred_cp[p] = ranks_num[14]
            else:
                pred_cp[p] = ranks_num[15]
        else:
            pred_cp[p]=0
    for t in range(len(tgt_test)):
        #gold
        if (((tgt_test[t] >= 505) and (tgt_test[t] <= 624.5)) and ((pred_cp[t] >= 505) and (pred_cp[t] <= 624.5))) or (((tgt_test[t] >= 875) and (tgt_test[t] <= 1035)) and ((pred_cp[t] >= 875) and (pred_cp[t] <= 1035))) or (((tgt_test[t] >= 1505) and (tgt_test[t] <= 1788)) and ((pred_cp[t] >= 1505) and (pred_cp[t] <= 1788))):
            corr+=1
            if data_type == 'a':
                all_cranks_train[alg][ranks_num.index(pred_cp[t])]+=((1/len(pred))*16)
            else:
                all_cranks_test[alg][ranks_num.index(pred_cp[t])]+=((1/len(pred))*16)
    if data_type == 'a':
        print(f"Average is {(sum(all_cranks_train[alg])/len(all_cranks_train[alg]))}")
    else:
        print(f"Average is {(sum(all_cranks_test[alg])/len(all_cranks_test[alg]))}")        
    return corr/len(tgt_test)

[ 505. ,  564.5,  624.5,  875. ,  955. , 1035. , 1505. , 1645. , 1788. ]

def r_predicts_to_acc(pred, tgts, alg, data_type):
    all_cranks_train = []
    all_cranks_test = []
    all_cranks_train.append([0]*16)
    all_cranks_train.append([0]*16)
    all_cranks_train.append([0]*16)
    all_cranks_train.append([0]*16)
    all_cranks_train.append([0]*16)
    all_cranks_test.append([0]*16)
    all_cranks_test.append([0]*16)
    all_cranks_test.append([0]*16)
    all_cranks_test.append([0]*16)
    all_cranks_test.append([0]*16)
    corr = 0
    pred_cp = list(pred)
    tgt_test = list(tgts[0])
    for p in range(len(pred)):
        if pred[p] >= 475:
            if (pred[p] < 535):
                pred_cp[p] = ranks_num[0]
            elif (pred[p] < 594):
                pred_cp[p] = ranks_num[1]            
            elif (pred[p] < 655):
                pred_cp[p] = ranks_num[2]
            elif (pred[p] < 715):
                pred_cp[p] = ranks_num[3]
            elif (pred[p] < 774):
                pred_cp[p] = ranks_num[4]
            elif (pred[p] < 835):
                pred_cp[p] = ranks_num[5]
            elif (pred[p] < 915):
                pred_cp[p] = ranks_num[6]
            elif (pred[p] < 995):
                pred_cp[p] = ranks_num[7]
            elif (pred[p] < 1075):
                pred_cp[p] = ranks_num[8]
            elif (pred[p] < 1195):
                pred_cp[p] = ranks_num[9]
            elif (pred[p] < 1315):
                pred_cp[p] = ranks_num[10]
            elif (pred[p] < 1435):
                pred_cp[p] = ranks_num[11]
            elif (pred[p] < 1575):
                pred_cp[p] = ranks_num[12]
            elif (pred[p] < 1715):
                pred_cp[p] = ranks_num[13]
            elif (pred[p] < 1861):
                pred_cp[p] = ranks_num[14]
            else:
                pred_cp[p] = ranks_num[15]
        else:
            pred_cp[p]=0
    for t in range(len(tgt_test)):       
        if (tgt_test[t] == pred_cp[t]):
            corr+=1
            if data_type == 'a':
                all_cranks_train[alg][ranks_num.index(pred_cp[t])]+=((1/len(pred))*16)
            else:
                all_cranks_test[alg][ranks_num.index(pred_cp[t])]+=((1/len(pred))*16)
    if data_type == 'a':
        print(f"Average is {(sum(all_cranks_train[alg])/len(all_cranks_train[alg]))}")
    else:
        print(f"Average is {(sum(all_cranks_test[alg])/len(all_cranks_test[alg]))}")        
    return corr/len(tgt_test)

def c_ranks_right(tgts, pred, alg, data_type):
    all_cranks_train = []
    all_cranks_test = []
    all_cranks_train.append([0]*16)
    all_cranks_train.append([0]*16)
    all_cranks_train.append([0]*16)
    all_cranks_train.append([0]*16)
    all_cranks_train.append([0]*16)
    all_cranks_test.append([0]*16)
    all_cranks_test.append([0]*16)
    all_cranks_test.append([0]*16)
    all_cranks_test.append([0]*16)
    all_cranks_test.append([0]*16)
    pred_cp = list(pred)
    tgt_test = list(tgts[0])
    for t in range(len(tgt_test)):
        if (tgt_test[t] == pred_cp[t]):
            if data_type == 'a':
                all_cranks_train[alg][ranks.index(pred_cp[t])]+=(1/2100)
            else:
                all_cranks_test[alg][ranks.index(pred_cp[t])]+=(1/900)
    return

In [7]:
#run this code to train and test the lasso model

#SelectKBest was attempted but did not improve performance, code used was the same as 
#the example at https://www.datatechnotes.com/2021/02/seleckbest-feature-selection-example-in-python.html

#Regression - LASSO (include r2, mse, mae)

alphas = 10**np.linspace(-5,-10,100)*40
#based on test, best alpha found is below
alph = 0.00015


las_model = Lasso(alpha=alph, max_iter=10000).fit(feat_train, r_tgt_train)
las_pred_train = las_model.predict(feat_train)
las_pred_test = las_model.predict(feat_test)
lacc_train = round(r_predicts_to_acc(las_pred_train, r_tgt_train, 0, 'a')*100, 2)
lacc_test = round(r_predicts_to_acc(las_pred_test, r_tgt_test, 0, 'b')*100, 2)
print(lacc_train, lacc_test)
print(r2_score(r_tgt_test, las_pred_test))
print(mean_squared_error(r_tgt_test, las_pred_test, squared=False))
print(mean_absolute_error(r_tgt_test, las_pred_test))

/Library/Python/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.162e+08, tolerance: 6.370e+05
  model = cd_fast.enet_coordinate_descent(


Average is 0.17658492822966454
Average is 0.17578124999999975
17.66 17.58
0.7432089571751109
221.16652212527268
172.47526691368694


In [4]:
#run this code to train and test the polynomial regression model

#Regression - Polynomial, takes a bit of time to load
#code from https://towardsdatascience.com/polynomial-regression-with-scikit-learn-what-you-should-know-bed9d3296f2
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

degree = 2
polyreg = make_pipeline(PolynomialFeatures(degree, include_bias=False, interaction_only=False, order='C'), LinearRegression())
polyreg.fit(feat_train, r_tgt_train)
p_pred_test = polyreg.predict(feat_test)
p_pred_train = polyreg.predict(feat_train)
pacc_test = round(r_predicts_to_acc(p_pred_test, r_tgt_test, 1, 'b')*100, 2)
pacc_train = round(r_predicts_to_acc(p_pred_train, r_tgt_train, 1, 'a')*100, 2)

#p_r2 = r2_score(r_tgt_test, p_pred)
#p_mse = mean_squared_error(r_tgt_test, p_pred)
#p_mae = mean_absolute_error(r_tgt_test, p_pred)
print(pacc_train, pacc_test)
print(r2_score(r_tgt_test, p_pred_test))
print(mean_squared_error(r_tgt_test, p_pred_test, squared=False))
print(mean_absolute_error(r_tgt_test, p_pred_test))

Average is 0.20431082589285676
Average is 0.21375598086124295
21.38 20.43
0.7040464054908577
237.43309993396156
155.5715447208643


In [8]:
#run this cell to train and test the random forest model

#Classification - Random Forest

#links below used to obtain code and inform methods of improving performance
#https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
#https://www.datasciencelearner.com/how-to-improve-accuracy-of-random-forest-classifier/
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
#https://www.datacamp.com/tutorial/random-forests-classifier-python
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score

n_estimators = [150, 200, 250]
max_features = ['sqrt', 'log2']
max_depth = [16, 32, None]
min_samples_split = [8, 10, 14]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap}
#reduce feature number
best_score = 0
best_i = 0
rf_model = RandomForestClassifier(n_estimators = 100, max_depth=4, n_jobs=-1, random_state=3)
#kf_model_scores = cross_val_score(rf_model, feat_train.append(feat_test), np.ravel(c_tgt_train.append(c_tgt_test)), cv=10)
#temp_best = max(kf_model_scores)
#print(f"{temp_best}, {i}, +{round((temp_best-best_score)*100, 2)}%")
#if temp_best > best_score:
#    best_i = i
#    best_score = temp_best
rf_model.fit(feat_train, np.ravel(c_tgt_train))
rf_pred_train = rf_model.predict(feat_train)
rf_pred_test = rf_model.predict(feat_test)
#converting to general labels for comparison to external paper
#for i in range(len(rf_pred_train)):
#    rf_pred_train[i] = rf_pred_train[i].split('-')[0]
#for j in range(len(rf_pred_test)):
#    rf_pred_test[j] = rf_pred_test[j].split('-')[0]
#for a in range(len(c_tgt_train)):
#    c_tgt_train.at[a, 0] = c_tgt_train[0][a].split('-')[0]
#for b in range(len(c_tgt_test)):
#    c_tgt_test.at[b, 0] = c_tgt_test[0][b].split('-')[0]
rfacc_train = round(metrics.accuracy_score(rf_pred_train, c_tgt_train)*100, 2)
rfacc_test = round(metrics.accuracy_score(rf_pred_test, c_tgt_test)*100, 2)
#c_ranks_right(c_tgt_train, rf_pred_train, 2, 'a')
#c_ranks_right(c_tgt_test, rf_pred_test, 2, 'b')
print(rfacc_train, rfacc_test)
print(metrics.f1_score(rf_pred_test, c_tgt_test, average='weighted'))
print(metrics.precision_score(rf_pred_test, c_tgt_test, average='weighted'))
print(metrics.recall_score(rf_pred_test, c_tgt_test, average='weighted'))

23.91 22.42
0.2523184918170106
0.36759341492944836
0.22419084821428573


In [6]:
#run this cell to train and test the xgboost model

#code obtained from https://www.kaggle.com/code/stuarthallows/using-xgboost-with-scikit-learn/notebook
#Classification - XGBoost 
#exact is best but takes a long time, use hist for quicker results   
import xgboost as xgb
import time
from sklearn import preprocessing, metrics

#reduce tree depth
#reduce number of trees
#lower learning rate
#change objective?
#reduce number of features
start = time.time()
enc = preprocessing.LabelEncoder()
xg_tgt_train = np.ravel(c_tgt_train)
xg_tgt_test = np.ravel(c_tgt_test)
enc.fit(xg_tgt_train)
enc.fit(xg_tgt_test)
c_tgt_train_enc = enc.transform(xg_tgt_train)
c_tgt_test_enc = enc.transform(xg_tgt_test)
xgb_model = xgb.XGBClassifier(n_estimators=150, max_depth=12, eta=0.01, objective="multi:softprob", tree_method='exact')
xgb_model.fit(feat_train, c_tgt_train_enc)
x_pred_train = xgb_model.predict(feat_train)
x_pred_train = enc.inverse_transform(x_pred_train)
x_pred_test = xgb_model.predict(feat_test)
x_pred_test = enc.inverse_transform(x_pred_test)
xacc_train = round(metrics.accuracy_score(x_pred_train, c_tgt_train)*100, 2)
xacc_test = round(metrics.accuracy_score(x_pred_test, c_tgt_test)*100, 2)
#c_ranks_right(c_tgt_train, x_pred_train, 3, 'a')
#c_ranks_right(c_tgt_test, x_pred_test, 3, 'b')
print(xacc_train, xacc_test)
end = time.time()
print(end-start)
print(metrics.f1_score(x_pred_test, c_tgt_test, average='weighted'))
print(metrics.precision_score(x_pred_test, c_tgt_test, average='weighted'))
print(metrics.recall_score(x_pred_test, c_tgt_test, average='weighted'))

KeyboardInterrupt: 

In [7]:
#run this cell to train and test the naive bayes model

#code from https://scikit-learn.org/stable/modules/naive_bayes.html
#Classification - Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

als = 10**np.linspace(-9,-15,100)
al = 7.05480231071863e-11 #best value found

nb_model = GaussianNB(var_smoothing=al)
nb_model.fit(feat_train, np.ravel(c_tgt_train))
nb_pred_train = nb_model.predict(feat_train)
nb_pred_test = nb_model.predict(feat_test)
nbacc_train = round(metrics.accuracy_score(nb_pred_train, c_tgt_train)*100, 2)
nbacc_test = round(metrics.accuracy_score(nb_pred_test, c_tgt_test)*100, 2)
#c_ranks_right(c_tgt_train, nb_pred_train, 4, 'a')
#c_ranks_right(c_tgt_test, nb_pred_test, 4, 'b')
print(nbacc_train, nbacc_test, al)
print(metrics.f1_score(nb_pred_test, c_tgt_test, average='weighted'))
print(metrics.precision_score(nb_pred_test, c_tgt_test, average='weighted'))
print(metrics.recall_score(nb_pred_test, c_tgt_test, average='weighted'))

22.14 21.51 7.05480231071863e-11
0.23403711055641493
0.28264298730966997
0.21505301339285715


In [ ]:
#Deep Learning code below

In [2]:
#this cell reads in the dataset from memory and should be
#exectued before any training is done

#reading in and formatting replays

import pandas as pd
import csv
import numpy as np
from sklearn.model_selection import train_test_split

ranks = ["gold-1", "gold-2", "gold-3", "platinum-1", "platinum-2", "platinum-3", "diamond-1", "diamond-2", "diamond-3", "champion-1", "champion-2", "champion-3", "grand-champion-1", "grand-champion-2", "grand-champion-3", "supersonic-legend"]
dif_ranks = ["gold-1", "gold-2", "gold-3", "diamond-1", "diamond-2", "diamond-3", "grand-champion-1", "grand-champion-2", "grand-champion-3"]
ranks_num = []
infile = open("MMR_branges.txt", "r+")
base_nums = []
for line in infile:
    base_nums.append(int((line.split('\n')[0]).split(',')[1]))
infile.close
for i in range(len(base_nums)-1):
    ranks_num.append((base_nums[i]+base_nums[i+1])/2)
d = [i for i in range(len(ranks_num))]
a, b, c = np.polyfit(d, ranks_num, 2)
ranks_num.append(a*(len(ranks_num)**2)+b*len(ranks_num)+c)
d.append(len(ranks_num)-1)
r_tgt_train = pd.DataFrame([])
r_tgt_test = pd.DataFrame([])
c_tgt_train = pd.DataFrame([])
c_tgt_test = pd.DataFrame([])
feat_train = pd.DataFrame([])
feat_test = pd.DataFrame([])

for r in ranks:
    temp_csv = pd.read_csv(f"MAIN_{r}_replays.csv", dtype=str)
    feat_temp = temp_csv.drop("Unnamed: 0", axis=1)
    feat_temp = feat_temp.dropna()
    #2986 is min number of replays for each rank, this line balances the dataset
    feat_temp = feat_temp[:2986]
    rows = len(temp_csv.index)
    r_list_train = pd.DataFrame([ranks_num[ranks.index(r)] for i in range(int(rows*0.7))])
    r_list_test = pd.DataFrame([ranks_num[ranks.index(r)] for i in range(rows-int(rows*0.7))])
    c_list_train = pd.DataFrame(r for i in range(int(rows*0.7)))
    c_list_test = pd.DataFrame(r for i in range(rows-int(rows*0.7)))
    feat_train_temp, feat_test_temp = train_test_split(feat_temp, train_size=0.7, random_state=1)
    r_tgt_train = pd.concat([r_tgt_train, r_list_train])
    r_tgt_test = pd.concat([r_tgt_test, r_list_test])
    c_tgt_train = pd.concat([c_tgt_train, c_list_train])
    c_tgt_test = pd.concat([c_tgt_test, c_list_test])
    feat_train = pd.concat([feat_train, feat_train_temp])
    feat_test = pd.concat([feat_test, feat_test_temp])
    feat_train.reset_index(drop=True, inplace=True)
    feat_test.reset_index(drop=True, inplace=True)
    c_tgt_train.reset_index(drop=True, inplace=True)
    c_tgt_test.reset_index(drop=True, inplace=True)
    r_tgt_train.reset_index(drop=True, inplace=True)
    r_tgt_test.reset_index(drop=True, inplace=True)

for r in feat_train.index:
    feat_train['season'][r] = float(feat_train['season'][r][1])
    if 'T' in feat_train['core.mvp'][r]:
        feat_train.at[r, 'core.mvp']=1
    else:
        feat_train.at[r, 'core.mvp']=0
    
for r in feat_test.index:
    feat_test['season'][r] = float(feat_test['season'][r][1])
    if 'T' in feat_test['core.mvp'][r]:
        feat_test.at[r, 'core.mvp']=1
    else:
        feat_test.at[r, 'core.mvp']=0

feat_train = feat_train.astype(float)
feat_test = feat_test.astype(float)

#dropping features that don't provide any new information
bad_feats = ["boost.count_collected_big", "boost.count_stolen_big", "boost.count_collected_small", "boost.count_stolen_small", "boost.percent_zero_boost", "boost.percent_full_boost", "boost.percent_boost_0_25", "boost.percent_boost_25_50", "boost.percent_boost_50_75", "boost.percent_boost_75_100", "movement.avg_powerslide_duration", "movement.percent_slow_speed", "movement.percent_boost_speed", "movement.percent_supersonic_speed", "movement.percent_ground", "movement.percent_low_air", "movement.percent_high_air", "positioning.percent_defensive_third", "positioning.percent_offensive_third", "positioning.percent_neutral_third", "positioning.percent_defensive_half", "positioning.percent_offensive_half", "positioning.percent_behind_ball", "positioning.percent_infront_ball", "positioning.percent_most_back", "positioning.percent_most_forward", "positioning.percent_closest_to_ball", "positioning.percent_farthest_from_ball"]
feat_train.drop(columns=bad_feats, inplace=True)
feat_test.drop(columns=bad_feats, inplace=True)

feat_train['rank'] = c_tgt_train
feat_test['rank'] = c_tgt_test

#manually shuffling features before model is trained
feat_test = feat_test.sample(frac=1).reset_index()
feat_train = feat_train.sample(frac=1).reset_index()

#re-splitting dataframe into features and target labels after shuffling
c_tgt_train = pd.DataFrame(feat_train['rank'])
feat_train.drop(columns=['index', 'rank'], inplace=True)
c_tgt_test = pd.DataFrame(feat_test['rank'])
feat_test.drop(columns=['index', 'rank'], inplace=True)

In [3]:
#this cell scales the data
#this speeds up the training process and should be run before the dataset
#the dataset is turned into a dataloader

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
feat_train = pd.DataFrame(scaler.fit_transform(feat_train))
feat_test = pd.DataFrame(scaler.transform(feat_test))

#pca = PCA()
#feat_train = pd.DataFrame(pca.fit_transform(feat_train))
#feat_test = pd.DataFrame(pca.transform(feat_test))

In [4]:
#this cell contains the different network architectures attempted and
#converts the dataset from a dataframe into a dataloader
#run this cell with either SimpleNetwork or SimpleConvNetwork before
#training either model

#this code is based on https://github.com/atapour/dl-pytorch/blob/main/4.Classifier/4_PyTorch_Programming_Classifier.ipynb, which is released under the LGPL License
#conv code from https://www.analyticsvidhya.com/blog/2019/10/building-image-classification-models-cnn-pytorch/

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from livelossplot import PlotLosses
import os

ranks = ['gold-1', 'gold-2', 'gold-3', 'platinum-1', 'platinum-2', 'platinum-3', 'diamond-1', 'diamond-2', 'diamond-3', 'champion-1', 'champion-2', 'champion-3', 'grand-champion-1', 'grand-champion-2', 'grand-champion-3', 'supersonic-legend']
ranks_dict = {}
for val, rank in enumerate(ranks):
    ranks_dict[rank] = val

def to_tensor_loader(train, test):
    train = torch.tensor(train.values)
    test = torch.tensor(test.values)
    train = DataLoader(train, batch_size=256, drop_last=True)
    test = DataLoader(test, batch_size=256, drop_last=True)
    return train, test

def cycle(iterable):
    while True:
        for x in iterable:
            yield x

#converting tgt dataframes to tensors
t_train_tgt = torch.zeros(len(feat_train))
t_test_tgt = torch.zeros(len(feat_test))
for i in range(len(feat_train)):
    t_train_tgt[i] = ranks_dict[c_tgt_train['rank'][i]]
for i in range(len(feat_test)):
    t_test_tgt[i] = ranks_dict[c_tgt_test['rank'][i]]
            
train_loader, test_loader = to_tensor_loader(feat_train, feat_test)
train_iterator = iter(cycle(train_loader))
test_iterator = iter(cycle(test_loader))

#defining simple network and optimiser

class SimpleNetwork(nn.Module):
    def __init__(self):
        super(SimpleNetwork, self).__init__()
        self.l1 = nn.Linear(57, 47)
        self.l2 = nn.Linear(47, 37)
        self.l3 = nn.Linear(37, 27)
        self.l4 = nn.Linear(27, 16)
        self.do = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.l1(x)
        x = torch.nn.functional.leaky_relu(x, negative_slope=0.2)
        self.do
        x = self.l2(x)
        x = torch.nn.functional.leaky_relu(x, negative_slope=0.2)
        self.do
        x = self.l3(x)
        x = torch.nn.functional.leaky_relu(x, negative_slope=0.2)
        self.do
        x = self.l4(x)
        return x
    
class SimpleConvNetworkSeq(nn.Module):
    def __init__(self):
        super(SimpleConvNetworkSeq, self).__init__()
        self.cnn_layers = nn.Sequential(
            nn.Conv1d(1, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(4, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(56, 16)
        )
    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x
                
        
class SimpleConvNetwork(nn.Module):
    def __init__(self):
        super(SimpleConvNetwork, self).__init__()
        self.l1 = nn.Conv1d(1, 4, kernel_size=3, stride=1, padding=1) 
        self.l2 = nn.BatchNorm1d(4) 
        self.l3 = nn.Conv1d(4, 8, kernel_size=3, stride=1, padding=1) 
        self.l4 = nn.BatchNorm1d(8)
        self.do = nn.Dropout(0.2)
        self.l5 = nn.Linear(456, 228)
        self.l6 = nn.Linear(228, 114)
        self.l7 = nn.Linear(114, 57)
        self.l8 = nn.Linear(57, 16)
        
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = nn.functional.leaky_relu(x)
        x = self.l3(x)
        x = self.l4(x)
        self.do
        x = nn.functional.leaky_relu(x)
        x = x.view(x.size(0), -1)
        x = self.l5(x)
        x = nn.functional.leaky_relu(x)
        x = self.l6(x)
        x = nn.functional.leaky_relu(x)
        x = self.l7(x)
        x = nn.functional.leaky_relu(x)
        x = self.l8(x)
        return x
    
device = torch.device('cpu')
model = SimpleNetwork().to(device)
#model = SimpleConvNetwork().to(device)
optimiser = torch.optim.Adam(model.parameters(), lr=0.00001)
criterion=nn.CrossEntropyLoss()
liveloss=PlotLosses()
logs={}

In [5]:
#code below is used to train the neural network and monitor its performance
#run this cell to do this

#using optuna to improve performance https://towardsdatascience.com/hyperparameter-tuning-of-neural-networks-with-optuna-and-pytorch-22e179efc837
import time
#import optuna
start = time.time()

#def objective(trial):
#    params = {
#        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
#        'optimiser': trial.suggest_categorical('optimiser', ["Adam", "RMSprop", "SGD"]),
#        'n_unit': trial.suggest_int("n_unit", 28, 42)
#    }
#    model=build_model(params)
#    accuracy = train_and_evaluate(params, model)
#    return accuracy

#def build_model(params):
#    in_features=57
#    return nn.Sequential(
#        nn.Linear(in_features, params['n_unit']),
#        nn.LeakyReLU(),
#        nn.Linear(params['n_unit'], 16),
#        nn.LeakyReLU()
#    )

#def train_and_evaluate(param, model):

# outer loop - going over the steps we want to train for
step=0
while step <= 2000:
    train_accuracy=0
    all_preds = []
    all_tgts = []
    # inner loop - iterating over the batches
    for i, batch in enumerate(train_loader):

        # getting the input and the ground truth labels from the training set
        x, gt = batch, t_train_tgt[(i*256):((i+1)*256)]
        x, gt = x.to(device), gt.to(device)

        # forward pass
        x = x.view(x.size(0), -1)
        x = x.to(torch.float32)
        #x.unsqueeze_(1)
        output = model(x)

        # loss
        loss = criterion(output, gt.long())
        #optmiser = getattr(torch.optim, param['optimiser'])(model.parameters(), lr=param['learning_rate'])

        # explicitly set the gradients to zero before backpropagation
        model.zero_grad()

        # backward pass
        loss.backward()
        optimiser.step()

        # calculating the accuracy
        _, argmax = torch.max(output, dim=1)
        train_corr = argmax.eq(gt).float().mean() * 256
        train_accuracy+=train_corr
    
    train_accuracy/=(len(feat_train)-(len(feat_train)%256))
    train_accuracy*=100
    step+=1
    # every k steps we evaluate the model on the test set
    k=20
    if step % k == 0:
        test_accuracy=0
        # when we test, we don't need gradients
        with torch.no_grad():

            # beginning the loop for evaluation:
            for j, test_batch in enumerate(test_loader):
                # getting the input and the ground truth labels from the training set
                x, gt = test_batch, t_test_tgt[j*256:((j+1)*256)]
                x, gt = x.to(device), gt.to(device)

                # forward pass
                x = x.view(x.size(0), -1)
                x = x.to(torch.float32)
                #x.unsqueeze_(1)
                output = model(x)

                # test loss
                test_loss = criterion(output, gt.long())

                # calculating the test accuracy
                _, argmax = torch.max(output, dim=1)
                test_corr = argmax.eq(gt).float().mean() * 256
                test_accuracy+=test_corr

            test_accuracy/=(len(feat_test)-(len(feat_test)%256))
            test_accuracy*=100

            # logging train and test losses and accuracies
            logs['Loss'] = loss.item()
            logs['val_Loss'] = test_loss.item()
            logs['Accuracy'] = train_accuracy
            logs['val_Accuracy'] = test_accuracy
            print(f"Train: {round(float(train_accuracy), 4)} Test: {round(float(test_accuracy), 4)} Train Loss: {round(float(loss.item()), 4)} Test Loss: {round(float(test_loss.item()), 4)} Epoch: {step}")
            #liveloss.update(logs)
            #liveloss.send()
    #return test_accuracy



#study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
#study.optimize(objective, n_trials=30)
end = time.time()
overall = (end-start)/60
print(f"Took {overall} mins.")

Train: 12.3498 Test: 12.207 Train Loss: 2.5362 Test Loss: 2.5486 Epoch: 20
Train: 15.5829 Test: 15.8482 Train Loss: 2.296 Test Loss: 2.3049 Epoch: 40
Train: 18.2091 Test: 18.8267 Train Loss: 2.1831 Test Loss: 2.1809 Epoch: 60
Train: 20.8474 Test: 21.1984 Train Loss: 2.094 Test Loss: 2.0752 Epoch: 80


KeyboardInterrupt: 